# Word2vec 모델 구축
* * *
20대 및 21대 법안 텍스트를 이용한 word2vec 모델 구축하기.    
법안별 제안이유 및 주요내용 텍스트를 사용함.

In [40]:
#필요한 라이브러리 로드
import re
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
from eunjeon import Mecab
from gensim.models import Word2Vec as w2v
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tqdm.pandas()
tagger = Mecab() 

C:\Users\user\anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [12]:
#법안 데이터 로드: 20대 전체 및 21대 법안(~2020년 11월)

bills = pd.read_csv('bills_total.csv')

In [13]:
bills.head(5)

,Unnamed: 0,comm_code,id,name,no,result,gubn,proc_date,proc_stage,propose_date,proposer_kind,summary
0,0,9700481,PRC_D1C9S0U9X2I3M1A4A1J0R3X5A2N9M6,중견기업 성장촉진 및 경쟁력 강화에 관한 특별법 일부개정법률안(유승희의원 등 11인),2022602,임기만료폐기,처리의안,2020-05-29,소관위심사,2019-09-23,의원,제안이유 및 주요내용 \r\n\r\n 2013년 말 「중견기업 성장촉진 및 경쟁력...
1,1,9700481,PRC_B1R7Q1Q2Z2X6Z1B0C2S5Z1P3C3I3J9,벤처기업육성에 관한 특별조치법 일부개정법률안(박맹우의원 등 10인),2011011,대안반영폐기,처리의안,2019-04-05,본회의불부의,2017-12-26,의원,"제안이유 및 주요내용\r\n\r\n 현행법은 벤처기업의 요건을 정하고 있고, 법에..."
2,2,9700481,PRC_C1I9D1I0W0G1Y1W7J2V6Y1L1A4C6L4,중소기업창업 지원법 일부개정법률안(정우택의원 등 10인),2022784,임기만료폐기,처리의안,2020-05-29,소관위심사,2019-10-01,의원,제안이유 및 주요내용 \r\n\r\n 현행법에 따르면 중소벤처기업부장관은 창업 저...
3,3,9700008,PRC_O1I8Y0J6P2X1O1Q7H4L7C0G9I7M0O5,독점규제 및 공정거래에 관한 법률 일부개정법률안(김병관의원 등 21인),2013979,임기만료폐기,처리의안,2020-05-29,소관위심사,2018-06-21,의원,제안이유 및 주요내용\r\n\r\n 벤처캐피탈(Venture Capital)은 벤...
4,4,9700300,PRC_S1J7X0K8L0S7U1I5U5U3N4V8C8V5H8,조세특례제한법 일부개정법률안(김수민의원 등 10인),2008412,대안반영폐기,처리의안,2018-05-21,대안반영폐기,2017-08-07,의원,제안이유 및 주요내용\r\n\r\n 중소기업청의 2013년 창업기업 실태조사에 따...


In [31]:
#법안별 주요내용 텍스트 전처리: 특수문자 등 삭제 + 형태소 분석

corpus=[] #doc2vec용 말뭉치
corpus_re=[] #word2vec용 말뭉치

for txt in tqdm(bills['summary']):
    try:
        txt = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', txt)
    except:
        txt = 'NaN'
    corpus.append(txt)  #정규식만 적용한 법안 텍스트 말뭉치    
    
    pos_text = tagger.pos(txt)
    word_list = []
    for word in pos_text:
        word_list.append(word[0])
    corpus_re.append(word_list) #단어 리스트의 리스트

100%|████████████████████████████| 28870/28870 [00:18<00:00, 1554.65it/s]


In [35]:
#모델구축
#벡터 사이즈는 20, 윈도우 20, 단어의 최소 등장 횟수 10으로 설정, 스킵그랩 사용(sg=1), 네거티브 샘플링(hs=0)

model = w2v(corpus_re, size=20, window=20, min_count=10, iter=20, sg=1, hs=0) 
model.save('w2v_bills.model') #모델 저장

In [36]:
#단어 벡터 구하기
model.wv['벤처']

array([-0.3615267 , -0.45887735, -0.78847545, -0.6981663 ,  1.2574933 ,
        0.46788543, -0.6257676 , -0.364747  ,  0.38551393,  0.01057009,
        0.05829025,  0.48117983,  0.357763  ,  0.24879684,  0.17585683,
       -0.04012185, -0.04546675,  0.911057  ,  0.21567512,  0.45867044],
      dtype=float32)

In [37]:
#유사도 높은 단어 구하기
print(model.wv.most_similar(positive=["벤처"], topn=10))

[('중소', 0.8784818649291992), ('창업', 0.8504295349121094), ('비대', 0.8434056043624878), ('기업', 0.8372299671173096), ('진출', 0.8324289917945862), ('실수요', 0.8220025300979614), ('창업자', 0.7870969176292419), ('한제', 0.7862508296966553), ('유치', 0.7851044535636902), ('리츠', 0.7782872915267944)]


# Doc2vec 모델 구축
* * *
문서 간 유사도를 보여주는 doc2vec 모델 구축.

In [41]:
#법안별 텍스트에 각각 tag 부여하기

tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(corpus)]

In [43]:
#doc2vec 모델 구축

model = Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.00025, min_count=1, dm =1)
model.build_vocab(tagged_data)  #vocabulary를 먼저 구축한다.

In [45]:
#학습시키기
for epoch in range(100):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    model.alpha -= 0.0002
    model.min_alpha = model.alpha

model.save("d2v_bills.model")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [48]:
#랜덤으로 고른 문서와 유사한 문서들이 나오는지 확인하기

doc_id = random.randint(0, len(corpus) - 1)
sims = model.docvecs.most_similar(str(doc_id))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ''.join(corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
print('MS Document ({}): «{}»\n'.format(sims[0][0], ''.join(corpus[int(sims[0][0])])))
print('MS Document ({}): «{}»\n'.format(sims[1][0], ''.join(corpus[int(sims[1][0])])))

Test Document (26742): «제안이유      현행 지방자치제도가 정착되고 활성화되어 지방의회의 역할이 증대되고 있음에도 불구하고  지방의회의 조직  권한 및 전문성은 집행기관에 비하여 취약하여 집행부와 의회 상호 간에 견제와 균형이 효과적으로 이루어지지 못하고 있음     특히 지방의회 사무직원의 임명권을 지방자치단체의 장이 가지고 있어 사무직원이 소신 있게 의회에 대한 지원 역할을 하는 데 한계가 있으며  지방의회 소관 세출예산의 편성권이 지방자치단체의 장의 전속권한으로 되어 있어 온전한 견제기능을 발휘하는 데 어려움이 있음     이에 지방의회 소속 직원에 대한 인사권과 지방의회 소관 세출예산편성권을 지방의회의 의장에게 부여함으로써 지방의회의 견제와 감시 기능을 강화하려는 것임       주요내용    가  지방의회 인사의 독립성을 강화하기 위하여 지방의회에 두는 사무직원의 임면권을 지방의회의 의장에게 부여함 안 제  조제 항    나  지방자치단체의 예산 결산 등 재정운용 관련 사항과 자치입법 및 정책 관련 사항을 조사 연구 분석 평가하고  관련 정보 및 자료를 제공하는 등 의정활동을 지원하기 위한 의정활동지원기구를 둘 수 있도록 함 안 제  조의 제 항 신설         참고사항      이 법률안은 정청래의원이 대표발의한  지방공무원법 일부개정법률안  의안번호 제   호  및  지방공무원 교육훈련법 일부개정법률안  의안번호 제   호 의 의결을 전제로 하는 것이므로 같은 법률안이 의결되지 아니하거나 수정의결되는 경우에는 이에 맞추어 조정되어야 할 것임 »

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d300,n5,w5,s0.001,t3):

MS Document (26575): «제안이유      현행 지방자치제도가 정착되고 활성화되어 지방의회의 역할이 증대되고 있음에도 불구하고  지방의회의 조직  권한 및 전문성은 집행기관에 비하여 취약하여 집행부와 의회 상호 간에 견제와 균형이 효과적으로 